In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import print_function
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, MaxPooling2D
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Nadam
from PIL import Image
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
import os

def preprocess_images(images):
    resized_images = []
    for img in images:
        resized_img = np.array(Image.fromarray(img).resize((320, 320)))
        if len(resized_img.shape) == 2:
            resized_img = np.expand_dims(resized_img, axis=-1)
            resized_img = np.repeat(resized_img, 3, axis=-1)
        resized_images.append(resized_img)
    resized_images = np.array(resized_images)
    resized_images = resized_images.astype('float32')
    resized_images = (resized_images - 127.5) / 127.5
    return resized_images

class DCGAN():
    def __init__(self):
        self.img_rows = 320
        self.img_cols = 320
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        optimizer_d = Nadam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer_d,
                                   metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        optimizer_g = Nadam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer_g)

        # The generator takes noise as input and generates images
        z = Input(shape=(rnd_len,))
        img = self.generator(z)

        # For the combined model, we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        optimizer_c = Nadam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer_c)

    def build_generator(self):
        model = Sequential()
        model.add(Dense(256 * 20 * 20, activation="relu", input_dim=rnd_len))
        model.add(Reshape((20, 20, 256)))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2DTranspose(256, kernel_size=3, strides=2, padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Conv2DTranspose(3, kernel_size=3, strides=1, padding='same'))
        model.summary()
        noise = Input(shape=(rnd_len,))
        img = model(noise)
        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)
        return Model(img, validity)

    def train(self, epochs, batch_size=12, save_interval=50):
        image_list = []
        for filename in glob.glob('/kaggle/input/dataset-juntado/dataset_JUNTADO/train/maligno/*.jpg'):
            im = Image.open(filename)
            reim = im.resize((320, 320), Image.ANTIALIAS)
            reim = np.array(reim)
            image_list.append(reim)

        image_stack = np.array(image_list)
        X_train = preprocess_images(image_stack)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (half_batch, rnd_len))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, rnd_len))
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.generate_and_save_images(epoch)

    def generate_and_save_images(self, epoch, rnd_len=100, save_model_path='/kaggle/working/saved_models/dcgan1.h5'):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, rnd_len))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        gen_imgs = np.uint8(255 * gen_imgs)
        os.makedirs("/kaggle/working/generated_images", exist_ok=True)
        fig, axs = plt.subplots(r, c, figsize=(12, 12))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("/kaggle/working/generated_images/samples_%d.png" % epoch, dpi=100)
        plt.close()
        self.generator.save(save_model_path)

if __name__ == '__main__':
    rnd_len = 100
    dcgan = DCGAN()
    time_in = time.time()
    dcgan.train(epochs=100001, batch_size=30, save_interval=200)
    time_out = time.time()
    print('\n', 'Time cost:', '\n', time_out - time_in)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 320, 320, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 320, 320, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 160, 160, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 160, 160, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 160, 160, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 160, 160, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 80, 80, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 80, 80, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 80, 80, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 80, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 40, 40, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 40, 40, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 40, 40, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 40, 40, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 102400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │       102,401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 490,817 (1.87 MB)

 Trainable params: 490,817 (1.87 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 102400)         │    10,342,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 20, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 40, 40, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 40, 40, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 40, 40, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 80, 80, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 80, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 80, 80, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 160, 160, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 160, 160, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 160, 160, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 320, 320, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 320, 320, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 320, 320, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 320, 320, 3)    │           867 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,323,587 (43.20 MB)

 Trainable params: 11,322,115 (43.19 MB)

 Non-trainable params: 1,472 (5.75 KB)

/tmp/ipykernel_34/4056915849.py:117: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  reim = im.resize((320, 320), Image.ANTIALIAS)
